In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from datetime import datetime

In [ ]:
mycolumns = ['date', 'press','title','content','link']
mydata = {
    'date' : date_list,
    'press' : press_list,
    'title' : title_list,
    'content' : content_list,
    'link' : link_list
}

In [ ]:
query = "비트코인"
sort = 1
s_date = "2013.04.01"
e_date = "2021.04.01"
s_from = "20130401"
e_to = "20210401"


mycolumns = ['date', 'press','title','content','link']

num = 1

while True : 
    page = 1
    file_name = "/Users/jaehyun/Desktop/psat_data/news_data_" + str(num) + ".csv"
    
    if num>=2 : 
        year = int(date_list[-1][0:4] + date_list[-1][5:7] + date_list[-1][8:10]) #YYYYMMDD형태
        if year <= 20130403 : 
            break
            
        s_date, s_from = '2013.04.01', '20130401'
        e_date, e_to = date_list[-1][:-1], str(year)
        
    title_list = [] # 제목 모음 리스트
    content_list = [] # 내용 모음 리스트
    link_list = [] # 링크 모음 리스트
    date_list = [] # 날짜 모음 리스트
    press_list = [] # 언론사 모음 리스트
    
    while page <= 3991 :  # 400페이지까지 반복
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + query + "&sort="+str(sort)+"photo=0&field=0&pd=3&ds=" + s_date + "&de=" + e_date + "&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:dd,p:from" + s_from + "to" + e_to +",a:all&start=" + str(page)
        #최신순 : sort = 1, 관련도순 : sort = 0 , 오래된 순 : sort = 2
        response = requests.get(url)
        html = response.text
    
        soup = BeautifulSoup(html,'html.parser')
        li_tmp = soup.find_all('li',{'class' : 'bx'})
    
        for i in li_tmp : 
            try : 
                title = i.find('a',{'class' : 'news_tit'}).text #제목
                content = i.find('div',{'class' : 'news_dsc'}).text #내용
                link = i.find('a',{'class' : 'news_tit'})['href'] #링크
                press = i.find('a',{'class' : 'info press'}).text #언론사
            
                date_set = i.findAll('span',{'class' : 'info'})#날짜
                if date_set : 
                    if 'A' in date_set[0].text or '면' in date_set[0].text :
                        date = date_set[1].text
                    else : 
                        date = date_set[0].text
        
                title_list.append(title)
                content_list.append(content)
                link_list.append(link)
                press_list.append(press)
                date_list.append(date)
        
            except : 
                continue
    
        page += 10
        
    mydata = {
    'date' : date_list,
    'press' : press_list,
    'title' : title_list,
    'content' : content_list,
    'link' : link_list
    }
    
    my_df = pd.DataFrame(data = mydata, columns = mycolumns)
    my_df.to_csv(file_name, encoding = 'utf-8') # 파일을 나눠서 저장 - 중간에 끊길 수가 있으므로
    
    num += 1
    time.sleep(5)

In [ ]:
df = pd.read_csv("/Users/jaehyun/Desktop/psat_data/news_data_1.csv")

for i in range(2,103) : # 저장한 파일들을 불러들이는 과정, 총 103개의 파일이 생성됨
    file_name = "/Users/jaehyun/Desktop/psat_data/news_data_"+ str(i) +".csv"
    concat_df = pd.read_csv(file_name)
    df = pd.concat([df,concat_df]).drop_duplicates('content')
    df = df.reset_index(drop = True)

df = df.drop(['Unnamed: 0'],axis = 1)
df.to_csv("/Users/jaehyun/Desktop/psat_data/news_data_final.csv", encoding = 'utf-8')